In [1]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from dataloader import get_sar_dataloader
from sarpyx.processor.core.focus import CoarseRDA, ifft2d
import numpy as np

file_path = "/Data/sar_focusing/s1a-s1-raw-s-hh-20240517t151240-20240517t151255-053912-068d91.zarr"
filename = os.path.basename(file_path)
start_row= 0
patch_size = (1, -1)
start_column = 0
verbose = True


import zarr
store = zarr.open(file_path, mode='r')
echo_data, rc_data = store['raw'][0, :].reshape([1, -1]), store['rc'][0, :].reshape([1, -1])
#print(f"Echo data: {echo_data.shape}, RC data shape: {rc_data.shape}")
end_row = start_row + patch_size[0]
#metadata, ephemeris = loader.dataset.get_metadata(zfile=file_path, rows=slice(start_row, end_row))
metadata = pd.DataFrame(store.attrs['metadata']['data']).iloc[[0]]
ephemeris = pd.DataFrame(store.attrs['ephemeris']['data']).iloc[[0]]
#print(f"Ephemeris: {ephemeris}, Metadata: {metadata}")
raw_data = {'echo': echo_data, 'metadata': metadata, 'ephemeris': ephemeris}
#print(f'📊 Sliced raw data shape: {raw_data["echo"].shape}')
processor = CoarseRDA(
    raw_data=raw_data,
    verbose=verbose,
)
#print(f'🛠️ Processor initialized with raw data of shape: {raw_data["echo"].shape}')

# Focus the data
processor.fft2d()
processor.range_compression()
rc_result = ifft2d(processor.radar_data.copy())

print("=== COMPARISON RESULTS ===")
print(f"Original raw sample: {echo_data}")
print(f"RDA focusing produced these data: {rc_result}")
print(f"Actual range compressed sample: {rc_data}")

diff = rc_result - rc_data[0, :]  # Compare 1D arrays
max_diff = np.max(np.abs(diff))
mean_diff = np.mean(np.abs(diff))

print(f"\nDifference Analysis:")
print(f"  Maximum difference: {max_diff:.2e}")
print(f"  Mean difference: {mean_diff:.2e}")

if max_diff < 1e-10:
    print("✅ Results match within numerical precision!")
elif max_diff < 1e-6:
    print("⚠️  Results match within acceptable tolerance")
else:
    print("❌ Significant differences detected")

/Data/gdaga/anaconda3/envs/sar_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Memory efficient mode: enabled
Loaded radar data with shape: (1, 25716)
Azimuth lines: 1, Range lines: 25716
Data type: complex64
Memory usage: 0.00 GB
Generating transmission replica...
Range decimation code: 1
Range sample frequency: 100092592.64 Hz
TX pulse start frequency: -43801345.00 Hz
TX ramp rate: 1927378686407.00 Hz/s
TX pulse length: 0.000045 s
Number of TX values: 4549
Phase parameters - phi1: -460.87, phi2: 9.64e+11
Replica length: 4549
Transmission replica generated successfully!
FFT input data shape: (1, 25716)
RAM memory usage: 4.2%
Original radar data shape: (1, 25716)
Performing FFT along range dimension (axis=1)...
First FFT along range dimension completed, shape: (1, 25716)
RAM memory usage: 4.2%
Performing FFT along azimuth dimension (axis=0) with fftshift...
Second FFT along azimuth dimension completed, shape: (1, 25716)
RAM memory usage: 4.2%
FFT output data shape: (1, 25716)
- FFT performed successfully!
RAM memory usage: 4.2%
Memory usage: 4.2% -> 4.2% (Δ+0.0%)

In [2]:
import sys
import os
from numpy import copy
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from dataloader import get_sar_dataloader
from sarpyx.processor.core.focus import CoarseRDA, ifft2d

file_path = "/Data/sar_focusing/s1a-s1-raw-s-hh-20240517t151240-20240517t151255-053912-068d91.zarr"
filename = os.path.basename(file_path)
start_row= 0
patch_size = (1, -1)
start_column = 0
verbose = True

loader = get_sar_dataloader(
    data_dir="/Data/sar_focusing",
    level_from="raw",
    level_to="rc",
    batch_size=1,
    num_workers=0,
    patch_mode="rectangular", 
    patch_size = (1, -1),
    buffer = (0, 0),
    stride = (1, 300),
    shuffle_files = False,
    patch_order="row", 
    complex_valued = True,
    save_samples = False, 
    backend="zarr", 
    verbose=True, 
    samples_per_prod = 1000,
    cache_size = 1000, 
    online = True, 
    positional_encoding=False,
    file_pattern=filename,

    max_products=10000
)

echo_data, rc_data = loader.dataset[(file_path, start_row, start_column)]
echo_data = echo_data.numpy()
rc_data = rc_data.numpy()

#print(f"Echo data: {echo_data.shape}, RC data shape: {rc_data.shape}")
end_row = start_row + patch_size[0]
metadata, ephemeris = loader.dataset.get_metadata(zfile=file_path, rows=slice(start_row, end_row))

#print(f"Ephemeris: {ephemeris}, Metadata: {metadata}")
raw_data = {'echo': echo_data, 'metadata': metadata, 'ephemeris': ephemeris}
#print(f'📊 Sliced raw data shape: {raw_data["echo"].shape}')
processor = CoarseRDA(
    raw_data=raw_data,
    verbose=verbose,
)
#print(f'🛠️ Processor initialized with raw data of shape: {raw_data["echo"].shape}')

# Focus the data
print(f"Raw data before fft: {echo_data}")
processor.fft2d()
print(f"Raw data after fft: {processor.radar_data}")
print(f"Range filter used: {processor.get_range_filter()}")
processor.range_compression()
print(f"Range line after compression before ifft: {processor.radar_data}")
rc_result = ifft2d(processor.radar_data.copy())

print("=== COMPARISON RESULTS ===")
print(f"Original raw sample: {echo_data}")
print(f"RDA focusing produced these data: {rc_result}")
print(f"Actual range compressed sample: {rc_data}")

diff = rc_data - rc_result[0, :]  # Compare 1D arrays
max_diff = np.max(np.abs(diff))
mean_diff = np.mean(np.abs(diff))

print(f"\nDifference Analysis:")
print(f"  Maximum difference: {max_diff:.2e}")
print(f"  Mean difference: {mean_diff:.2e}")

if max_diff < 1e-10:
    print("✅ Results match within numerical precision!")
elif max_diff < 1e-6:
    print("⚠️  Results match within acceptable tolerance")
else:
    print("❌ Significant differences detected")

Found 396 files in the remote repository: '['s1a-s1-raw-s-hh-20230508t121142-20230508t121213-048442-05d3c0.zarr', 's1a-s1-raw-s-hh-20230731t121147-20230731t121217-049667-05f8f1.zarr', 's1a-s1-raw-s-hh-20240103t121147-20240103t121217-051942-0646ac.zarr', 's1a-s1-raw-s-hh-20240130t151239-20240130t151254-052337-06541b.zarr', 's1a-s1-raw-s-hh-20240517t151240-20240517t151255-053912-068d91.zarr', 's1a-s1-raw-s-hh-20240626t053141-20240626t053207-054490-06a193.zarr', 's1a-s1-raw-s-hh-20240825t053115-20240825t053148-055365-06c086.zarr', 's1a-s1-raw-s-hh-20240830t121144-20240830t121215-055442-06c352.zarr', 's1a-s1-raw-s-hh-20241005t121146-20241005t121216-055967-06d7fb.zarr', 's1a-s1-raw-s-hh-20241008t151239-20241008t151255-056012-06d9c9.zarr', 's1a-s1-raw-s-hh-20241125t151237-20241125t151253-056712-06f59c.zarr', 's1a-s1-raw-s-hh-20241219t151235-20241219t151251-057062-07037d.zarr', 's1a-s1-raw-s-hh-20250109t121141-20250109t121211-057367-070f7e.zarr', 's1a-s1-raw-s-hh-20250124t002434-20250124t0025

In [4]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import numpy as np
import os
from dataloader import get_sar_dataloader
from sarpyx.processor.core.focus import CoarseRDA, ifft2d

# Reference constants
F_REF = 37.53472224e6  # Reference frequency in Hz (37.53472224 MHz)

def range_dec_to_sample_rate(rgdec_code: int) -> float:
    """Convert range decimation code to sample rate in Hz."""
    lookup_table = {
        0: 3 * F_REF,
        1: (8/3) * F_REF,
        3: (20/9) * F_REF,
        4: (16/9) * F_REF,
        5: (3/2) * F_REF,
        6: (4/3) * F_REF,
        7: (2/3) * F_REF,
        8: (12/7) * F_REF,
        9: (5/4) * F_REF,
        10: (6/13) * F_REF,
        11: (16/11) * F_REF
    }
    
    if rgdec_code not in lookup_table:
        raise ValueError(f'Invalid range decimation code: {rgdec_code}')
    
    return lookup_table[rgdec_code]

def apply_window(signal: np.ndarray, window_type: str = 'hamming') -> np.ndarray:
    """Apply windowing function to signal."""
    if window_type.lower() == 'rectangular' or window_type is None:
        return signal
    
    signal_length = len(signal)
    
    if window_type.lower() == 'hamming':
        window = np.hamming(signal_length)
    elif window_type.lower() == 'hanning':
        window = np.hanning(signal_length)
    elif window_type.lower() == 'blackman':
        window = np.blackman(signal_length)
    else:
        raise ValueError(f'Unsupported window type: {window_type}')
    
    return signal * window

def get_reference_function(replica_chirp: np.ndarray, window_type: str = "hamming") -> np.ndarray:
    """Generate reference function from replica chirp following C++ implementation."""
    windowed_replica = apply_window(replica_chirp, window_type)
    reference = windowed_replica.copy()
    
    # Calculate energy normalization 
    norm = np.abs(windowed_replica)
    norm_size = len(norm)
    norm_modified = norm * norm / norm_size
    energy = np.sum(norm_modified)
    
    # Apply FFT and conjugate 
    reference = np.fft.fft(reference)
    reference = np.conjugate(reference)
    
    # Normalize by energy
    if energy > 0:
        reference = reference / energy
    else:
        print("Warning: ZERO ENERGY DETECTED, SKIPPING NORMALIZATION")
    
    return reference

def generate_tx_replica(metadata_row: dict) -> np.ndarray:
    """Generate transmission replica from metadata parameters."""
    # Extract parameters from metadata
    rgdec = int(metadata_row['range_decimation'])
    txpsf = metadata_row['tx_pulse_start_freq']
    txprr = metadata_row['tx_ramp_rate'] 
    txpl = metadata_row['tx_pulse_length']
    
    print(f"TX Parameters:")
    print(f"  Range decimation: {rgdec}")
    print(f"  TX pulse start freq: {txpsf:.2f} Hz")
    print(f"  TX ramp rate: {txprr:.2f} Hz/s")
    print(f"  TX pulse length: {txpl:.6f} s")
    
    # Calculate range sample frequency
    range_sample_freq = range_dec_to_sample_rate(rgdec)
    print(f"  Range sample freq: {range_sample_freq:.2f} Hz")
    
    # Generate replica
    num_tx_vals = int(txpl * range_sample_freq)
    tx_replica_time_vals = np.linspace(-txpl/2, txpl/2, num=num_tx_vals)
    
    phi1 = txpsf + txprr * txpl / 2
    phi2 = txprr / 2
    
    print(f"  Number of TX values: {num_tx_vals}")
    print(f"  Phase parameters - phi1: {phi1:.2f}, phi2: {phi2:.2e}")
    
    tx_replica = np.exp(
        2j * np.pi * (phi1 * tx_replica_time_vals + phi2 * tx_replica_time_vals**2)
    )
    return tx_replica

def range_compression_with_metadata(
    raw_data: np.ndarray, 
    metadata_row: dict, 
    window_type: str = "hamming",
    verbose: bool = True
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Perform range compression using metadata parameters.
    
    Args:
        raw_data: Raw SAR data line (1D array) - SHOULD BE THE RANGE LINE, NOT AZIMUTH LINE
        metadata_row: Dictionary containing metadata for single row
        window_type: Window type for processing
        verbose: Whether to print debug info
        
    Returns:
        Tuple of (range_compressed_data, range_filter, tx_replica)
    """
    if verbose:
        print(f"Starting range compression with metadata...")
        print(f"Raw data shape: {raw_data.shape}")
        print(f"Metadata keys: {list(metadata_row.keys())}")
    
    # CRITICAL FIX: Extract the range line properly
    if len(raw_data.shape) == 2:
        if raw_data.shape[0] == 1:
            # We have (1, N) - take the range line
            range_line = raw_data[0, :]
            print(f"✅ Extracted range line from shape {raw_data.shape} -> {range_line.shape}")
        else:
            # We have (N, 1) - this is wrong, should be range samples
            print(f"❌ ERROR: Expected (1, range_samples), got {raw_data.shape}")
            return None, None, None
    else:
        # Already 1D
        range_line = raw_data
    
    line_length = len(range_line)
    
    if verbose:
        print(f"Processing single range line with {line_length} samples")
    
    # Step 1: Generate TX replica from metadata
    tx_replica = generate_tx_replica(metadata_row)
    
    # Step 2: Extract range decimation and calculate sample frequency
    rgdec = int(metadata_row['range_decimation'])
    range_sample_freq = range_dec_to_sample_rate(rgdec)
    txpl = metadata_row['tx_pulse_length']
    
    num_tx_vals = int(txpl * range_sample_freq)
    
    if verbose:
        print(f"Processing parameters:")
        print(f"  Line length: {line_length}")
        print(f"  TX replica length: {len(tx_replica)}")
        print(f"  Num TX vals: {num_tx_vals}")
    
    # Step 3: Create range filter with zero padding
    range_filter = np.zeros(line_length, dtype=np.complex64)
    
    if line_length >= num_tx_vals:
        # Normal case: center the TX replica
        index_start = (line_length - num_tx_vals) // 2
        index_end = index_start + num_tx_vals
        range_filter[index_start:index_end] = tx_replica[:num_tx_vals]
        
        if verbose:
            print(f"  Placed replica at indices [{index_start}:{index_end}]")
    else:
        # Edge case: TX replica longer than raw data
        if verbose:
            print(f"  Warning: TX replica ({num_tx_vals}) longer than raw data ({line_length})")
        replica_start = (num_tx_vals - line_length) // 2
        replica_end = replica_start + line_length
        range_filter[:] = tx_replica[replica_start:replica_end]
    
    # Step 4: Generate normalized reference function
    range_filter = get_reference_function(range_filter, window_type=window_type)
    
    if verbose:
        print(f"Range filter generated: {range_filter}")
        print(f"  Range filter energy: {np.sum(np.abs(range_filter)**2):.6f}")
    
    # Step 5: Apply range compression
    # FFT the raw data
    if verbose:
        print(f"Range line before FFT: shape={range_line.shape}, first 5 samples: {range_line[:5]}")
    
    range_line_fft = np.fft.fft(range_line.copy())
    
    if verbose:
        print(f"Range line after FFT: shape={range_line_fft.shape}, first 5 samples: {range_line_fft[:5]}")
    
    # Multiply by range filter (matched filtering in frequency domain)
    compressed_fft = np.multiply(range_line_fft, range_filter)
    if verbose:
        print(f"Range line compressed before ifft: {compressed_fft}")
    # Inverse FFT to get range compressed data
    range_compressed_data = np.fft.ifftshift(np.fft.ifft(compressed_fft)) #np.fft.ifft(compressed_fft)
    
    if verbose:
        print(f"Range compression completed!")
        print(f"  Compressed data length: {len(range_compressed_data)}")
        print(f"  Peak amplitude: {np.max(np.abs(range_compressed_data)):.6f}")
    
    return range_compressed_data, range_filter, tx_replica

# ===== CORRECTED USAGE =====

file_path = "/Data/sar_focusing/s1a-s1-raw-s-hh-20240517t151240-20240517t151255-053912-068d91.zarr"
filename = os.path.basename(file_path)
start_row = 0
patch_size = (1, -1)  # 1 azimuth line, all range samples
start_column = 0
verbose = True

loader = get_sar_dataloader(
    data_dir="/Data/sar_focusing",
    level_from="raw",
    level_to="rc",
    batch_size=1,
    num_workers=0,
    patch_mode="rectangular", 
    patch_size=patch_size,
    buffer=(0, 0),
    stride=(1, 300),
    shuffle_files=False,
    patch_order="row", 
    complex_valued=True,
    save_samples=False, 
    backend="zarr", 
    verbose=True, 
    samples_per_prod=1000,
    cache_size=1000, 
    online=True, 
    positional_encoding=False,
    file_pattern=filename,
    max_products=10000
)

# Load data correctly
echo_data, rc_data_reference = loader.dataset[(file_path, start_row, start_column)]
echo_data = echo_data.numpy()
rc_data_reference = rc_data_reference.numpy()

print(f"✅ Loaded data shapes:")
print(f"   Echo data: {echo_data.shape}")  # Should be (1, 25716)
print(f"   RC reference: {rc_data_reference.shape}")  # Should be (1, 25716)

# Get metadata for this row
end_row = start_row + patch_size[0]
metadata, ephemeris = loader.dataset.get_metadata(zfile=file_path, rows=slice(start_row, end_row))
metadata_row = metadata.iloc[0].to_dict()

print("=== RANGE COMPRESSION WITH METADATA ===")

# Perform range compression using metadata - NOW WITH CORRECT DIMENSIONS
compressed_data, range_filter, tx_replica = range_compression_with_metadata(
    raw_data=echo_data,  # This is (1, 25716) - will be handled correctly
    metadata_row=metadata_row,
    window_type="hamming",
    verbose=True
)

print("\n=== COMPARISON RESULTS ===")
print(f"Generated TX replica length: {len(tx_replica)}")
print(f"Sebastian calculated range compressed line (first 5): {compressed_data[:5]}")
print(f"Zarr dataset range compressed line (first 5): {rc_data_reference[0, :5]}")  # Take first row

# Calculate difference - compare with correct dimensions
diff = compressed_data - rc_data_reference[0, :]  # Compare 1D arrays
max_diff = np.max(np.abs(diff))
mean_diff = np.mean(np.abs(diff))

print(f"\nDifference Analysis:")
print(f"  Maximum difference: {max_diff:.2e}")
print(f"  Mean difference: {mean_diff:.2e}")

if max_diff < 1e-10:
    print("✅ Results match within numerical precision!")
elif max_diff < 1e-6:
    print("⚠️  Results match within acceptable tolerance")
else:
    print("❌ Significant differences detected")

# Optional: Display some metadata that was used
print(f"\nMetadata used for processing:")
for key in ['range_decimation', 'tx_pulse_length', 'tx_pulse_start_freq', 'tx_ramp_rate']:
    if key in metadata_row:
        print(f"  {key}: {metadata_row[key]}")

Found 396 files in the remote repository: '['s1a-s1-raw-s-hh-20230508t121142-20230508t121213-048442-05d3c0.zarr', 's1a-s1-raw-s-hh-20230731t121147-20230731t121217-049667-05f8f1.zarr', 's1a-s1-raw-s-hh-20240103t121147-20240103t121217-051942-0646ac.zarr', 's1a-s1-raw-s-hh-20240130t151239-20240130t151254-052337-06541b.zarr', 's1a-s1-raw-s-hh-20240517t151240-20240517t151255-053912-068d91.zarr', 's1a-s1-raw-s-hh-20240626t053141-20240626t053207-054490-06a193.zarr', 's1a-s1-raw-s-hh-20240825t053115-20240825t053148-055365-06c086.zarr', 's1a-s1-raw-s-hh-20240830t121144-20240830t121215-055442-06c352.zarr', 's1a-s1-raw-s-hh-20241005t121146-20241005t121216-055967-06d7fb.zarr', 's1a-s1-raw-s-hh-20241008t151239-20241008t151255-056012-06d9c9.zarr', 's1a-s1-raw-s-hh-20241125t151237-20241125t151253-056712-06f59c.zarr', 's1a-s1-raw-s-hh-20241219t151235-20241219t151251-057062-07037d.zarr', 's1a-s1-raw-s-hh-20250109t121141-20250109t121211-057367-070f7e.zarr', 's1a-s1-raw-s-hh-20250124t002434-20250124t0025